In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Cust = set(range(1,3))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [3]:
transport = {i:10 for i in DC}
delivery = {(i,j):5 for i,j in Route}

dc_run = {(i,ct):100*ct for i in DC for ct in CT}
dc_stock = {i:1 for i in DC}
dc_ub = {ct:80*ct for ct in CT}

cust_stock = {j:1 for j in Cust}

demand = {(j,t):10 for j in Cust for t in Period}

In [4]:
model = Model()

In [5]:
X, x, y, sD, sC = {}, {}, {}, {}, {}

for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="C", name=f'X[{i},{t}]')

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

for i in DC:
    sD[i,0] = model.addVar(vtype="C", ub=0, name=f'sD[{i},{t}]')
    for t in Period:
        sD[i,t] = model.addVar(vtype="C", name=f'sD[{i},{t}]')

for j in Cust:
    sC[j,0] = model.addVar(vtype="C", ub=0, name=f'sC[{j},{t}]')
    for t in Period:
        sC[j,t] = model.addVar(vtype="C", name=f'sC[{j},{t}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons, DC_UB_Cons = {}, {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC) + sC[j,t-1]
            ==
            demand[j,t] + sC[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t_] for t_ in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t] + sD[i,t-1]
            ==
            quicksum(x[i,j,t] for j in Cust) + sD[i,t]
        )

for i in DC:
    for t in Period:
        DC_UB_Cons[i,t] = model.addConstr(
            X[i,t] + sD[i,t-1]
            <=
            quicksum(dc_ub[ct] * y[i,ct,t] for ct in  CT)
        )
    
model.update()

In [7]:
model.setObjective(
    quicksum(transport[i] * X[i,t] for i in DC for t in Period) +
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period) +
    quicksum(dc_stock[i] * sD[i,t] for i in DC for t in Period) +
    quicksum(cust_stock[j] * sC[j,t] for j in Cust for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [8]:
model.optimize()

     0     0 1320.00000    0    3 1780.00000 1320.00000  25.8%     -    0s
H    0     0                    1520.0000000 1320.00000  13.2%     -    0s
H    0     0                    1440.0000000 1320.00000  8.33%     -    0s
H    0     0                    1420.0000000 1320.00000  7.04%     -    0s
     0     0     cutoff    0      1420.00000 1420.00000  0.00%     -    0s

Cutting planes:
  Gomory: 3
  Implied bound: 2
  Network: 1

Explored 1 nodes (25 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 4: 1420 1440 1520 1780 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.420000000000e+03, best bound 1.420000000000e+03, gap 0.0000%


In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

X[1,1] 80.0
x[1,2,1] 40.0
x[1,1,1] 40.0
y[1,1,1] 1.0
sC[1,1] 30.000000000000036
sC[1,2] 20.000000000000036
sC[1,3] 9.999999999999986
sC[2,1] 30.000000000000036
sC[2,2] 20.000000000000036
sC[2,3] 9.999999999999986
